Prepare the Bunnies' Escape
===========================

You're awfully close to destroying the LAMBCHOP doomsday device and freeing Commander Lambda's bunny workers, but once they're free of the work duties the bunnies are going to need to escape Lambda's space station via the escape pods as quickly as possible. Unfortunately, the halls of the space station are a maze of corridors and dead ends that will be a deathtrap for the escaping bunnies. Fortunately, Commander Lambda has put you in charge of a remodeling project that will give you the opportunity to make things a little easier for the bunnies. Unfortunately (again), you can't just remove all obstacles between the bunnies and the escape pods - at most you can remove one wall per escape pod path, both to maintain structural integrity of the station and to avoid arousing Commander Lambda's suspicions. 

You have maps of parts of the space station, each starting at a work area exit and ending at the door to an escape pod. The map is represented as a matrix of 0s and 1s, where 0s are passable space and 1s are impassable walls. The door out of the station is at the top left (0,0) and the door into an escape pod is at the bottom right (w-1,h-1). 

Write a function solution(map) that generates the length of the shortest path from the station door to the escape pod, where you are allowed to remove one wall as part of your remodeling plans. The path length is the total number of nodes you pass through, counting both the entrance and exit nodes. The starting and ending positions are always passable (0). The map will always be solvable, though you may or may not need to remove a wall. The height and width of the map can be from 2 to 20. Moves can only be made in cardinal directions; no diagonal moves are allowed.

Languages
=========

To provide a Python solution, edit solution.py
To provide a Java solution, edit Solution.java

Test cases
==========
Your code should pass the following test cases.
Note that it may also be run against hidden test cases not shown here.

-- Python cases --

Input:
solution.solution([[0, 1, 1, 0], [0, 0, 0, 1], [1, 1, 0, 0], [1, 1, 1, 0]])
Output:
    7

Input:
solution.solution([[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 0], [0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 1, 1], [0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0]])
Output:
    11

-- Java cases --

Input:
Solution.solution({{0, 1, 1, 0}, {0, 0, 0, 1}, {1, 1, 0, 0}, {1, 1, 1, 0}})
Output:
    7

Input:
Solution.solution({{0, 0, 0, 0, 0, 0}, {1, 1, 1, 1, 1, 0}, {0, 0, 0, 0, 0, 0}, {0, 1, 1, 1, 1, 1}, {0, 1, 1, 1, 1, 1}, {0, 0, 0, 0, 0, 0}})
Output:
    11

In [217]:
class Point:
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y

    def incX(self):
        return Point(self.x + 1, self.y)

    def incY(self):
        return Point(self.x, self.y + 1)

    def decX(self):
        return Point(self.x - 1, self.y)

    def decY(self):
        return Point(self.x, self.y - 1)

    def isFinal(self, height, width):
        return self.x == height - 1 and self.y == width - 1

    def __repr__(self):
        return '(%d, %d)' % (self.x, self.y)

class DeadlockException(Exception):
    pass

class MapPathGenerator:
    def __init__(self, height, width, m):
        self.height = height
        self.width = width
        self.m = m
        self.route = []
        for i in range(height):
            row = []
            for j in range(width):
                row.append(0)
            self.route.append(row)

    def run(self):
        def checkPoint(point, step):
            return point.x < self.height and point.x >= 0 and \
                   point.y < self.width and point.y >= 0 and \
                   self.route[point.x][point.y] == 0 and \
                   self.m[point.x][point.y] == 0
        
        def finished(points):
            return any(point.isFinal(self.height, self.width) for point in points)

        directions = [Point.incX, Point.incY, Point.decX, Point.decY]
        point = Point()
        step = 1
        self.route[point.x][point.y] = step
        points = [point]
        while not finished(points):
            points_new = []
            step_new = step + 1
            for point in points:
                for direction in directions:
                    point_new = direction(point)
                    if checkPoint(point_new, step_new):
                        points_new.append(point_new)
                        self.route[point_new.x][point_new.y] = step_new

            if len(points_new) == 0:
                raise DeadlockException()

            points = points_new
            step = step_new
        #print (self.route)
        #print ('final points=', points)
        #assert len(points) == 1 # one final point

    def getMinPathLength(self):
        return self.route[self.height - 1][self.width - 1]

class BaseMap:
    def __init__(self, map):
        self.height = len(map)
        self.width = len(map[0])
        #assert 2 <= self.height <= 20
        #assert 2 <= self.width <= 20

        self.map = map

    def getMinPathLength(self):
        generator = MapPathGenerator(self.height, self.width, self.map)
        try:
            generator.run()
            return generator.getMinPathLength()
        except DeadlockException:
            return None


class ExtendedMap(BaseMap):
    def __init__(self, map):
        BaseMap.__init__(self, map)
        self.walls = []
        self.current_wall_index = 0
        for i in range(self.height):
            for j in range(self.width):
                if self.map[i][j] == 1:
                    self.walls.append(Point(i, j))

    def removeWall(self):
        while self.current_wall_index < len(self.walls):
            if self.current_wall_index > 0:
                prev_wall_index = self.current_wall_index - 1
                prev_wall = self.walls[prev_wall_index]
                self.map[prev_wall.x][prev_wall.y] = 1
            current_wall = self.walls[self.current_wall_index]
            self.map[current_wall.x][current_wall.y] = 0
            self.current_wall_index += 1
            yield self

    def getMinPathLength(self):
        len = BaseMap.getMinPathLength(self)
        min_len = None
        if len is not None:
            min_len = len
        for map_reduced in self.removeWall():
            len = map_reduced.getMinPathLength()
            if len is not None and (min_len is None or min_len is not None and len < min_len):
                min_len = len
        return min_len
    
def solution(map):
    extended_map = ExtendedMap(map)
    min_len = extended_map.getMinPathLength()
    assert min_len is not None
    return min_len

In [218]:
# Tests for empty maps
results = 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39
for size in range(2, 21):
    test = [[0]*size] * size
    assert solution(test) == results[size-2]

In [219]:
# Tests
test1 = ([0, 1],
         [0, 0]), 3

test2 = ([0, 0],
         [1, 0]), 3

test3 = ([0, 1],
         [1, 0]), 3

test4 = ([0, 1],
         [0, 0],
         [1, 0]), 4

test5 = ([0, 1, 1, 0],
         [0, 0, 0, 1],
         [1, 1, 0, 0]), 6

test6 = ([0, 1, 1, 0],
         [0, 0, 0, 1],
         [1, 1, 0, 0]), 6

test7 = ([0, 0, 0, 1],
         [0, 1, 0, 0],
         [1, 1, 0, 1],
         [1, 0, 0, 0]), 7

test8 = ([1, 0, 0, 1],
         [0, 1, 0, 0],
         [1, 1, 0, 1],
         [1, 0, 0, 0]), 7

test9 = ([0, 1, 1, 0], 
         [0, 0, 0, 1], 
         [1, 1, 0, 0], 
         [1, 1, 1, 0]), 7

test10 = ([0, 0, 0, 0, 0, 0], 
         [1, 1, 1, 1, 1, 0], 
         [0, 0, 0, 0, 0, 0], 
         [0, 1, 1, 1, 1, 1], 
         [0, 1, 1, 1, 1, 1], 
         [0, 0, 0, 0, 0, 0]), 11

test11 = ([0, 0, 0, 0, 0, 0], 
          [1, 1, 1, 1, 1, 0], 
          [1, 0, 0, 0, 0, 0], 
          [0, 1, 1, 1, 1, 1], 
          [0, 1, 1, 1, 1, 1], 
          [0, 0, 0, 0, 0, 0]), 21

test12 = ([0, 1, 0, 0, 0, 0], 
          [1, 1, 1, 1, 1, 0], 
          [1, 0, 0, 0, 0, 0], 
          [0, 1, 1, 0, 0, 0], 
          [0, 1, 1, 0, 1, 1], 
          [0, 0, 0, 0, 0, 0]), 15

test13 = ([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], 
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
          [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 39

tests = test1, test2, test3, test4, test5, test6, test7, test8, test9, test10, test11, test12, test13

for test in tests:
    assert solution(test[0]) == test[1]

In [220]:
# Standalone tests
test = ([0, 1, 1, 0, 0, 1, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0]), 13

tests = test,

for test in tests:
    #print (solution(test[0]))
    assert solution(test[0]) == test[1]